> **Jupyter slideshow:** This notebook can be displayed as slides. To view it as a slideshow in your browser, type the following in the console:


> `> ipython nbconvert [this_notebook.ipynb] --to slides --post serve`


> To toggle off the slideshow cell formatting, click the `CellToolbar` button, then `View --> Cell Toolbar --> None`.

<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

## Intro to Data Representation and Data Cleaning

_Authors: Dave Yerrington (SF)_

---

<img src="https://snag.gy/ywU34V.jpg" width="250">

### Learning Objectives
*After this lesson, you will be able to:*
- Inspect data types.
- Clean up a column using `df.apply()`.
- Recognize situations in which to use `.value_counts()` in your code.

### Lesson Guide

- [Data Quality Measures](#data_quality_measures)
- [Common Data Cleaning Strategies](#common_strategies)
- [`pandas` Tools for Cleaning Data](#cleaning_tools)
- [Common Operations on Data by Type](#common_operations)
- [Guided Practice: Inspecting Data Types and Applying Functions](#guided_practice)
- [Independent Practice: Sales Data](#independent_pratice)


<a id='common_strategies'></a>

### Common Data Cleaning Strategies

---

 - Remove missing values.
 - Remove incorrect values.
 - Update incorrect values.
  - Removing invalid characters.
  - Truncating part of a value.
  - Adding an extra numeral or string-based data.
 - Imputate missing or invalid data.
  - Calculating the mean/median/mode of a column, sometimes within group subsets.
  - Implementing model-based imputation (K-Nearest Neighbors, MICE, etc.).
 - Backfill or forward fill.


<a id='data_quality_measures'></a>

### Measures of Data Quality

---

 - What is the relative value of the data column?
 - Is the data encoded properly?
 - Is the data consistently encoded? Does it represent the information it contains appropriately?

<a id='cleaning_tools'></a>

### `pandas` Tools for Cleaning Data

---

We're starting to get more comfortable with using `pandas` for manipulating and examining data. Now, let's add a couple more tools to our toolbox.

The main data types in `pandas` objects are:
- `float`
- `int`
- `bool`
- `datetime64`
- `timedelta`
- `category`
- `object`

It is always important to evaluate the data types of columns to ensure that the information they contain is properly represented.

See [`pandas`: dtypes](http://pandas.pydata.org/pandas-docs/stable/pandas.pdf) for a more detailed reference.

We will be using two tools extensively in this lesson:

**The `.apply()` function**

This built-in function will apply a function to all cells, rows, or columns within a DataFrame. We will explore this process in detail below.

**The Series `.value_counts` attribute**

`pandas` Series objects have `.value_counts` attributes that return a new Series containing the counts of the data's unique values. This Series will be formatted in descending order by default, so the first element is the most frequently occuring value.

Note: `.value_counts` excludes the counts of null values in the column!

See [`pandas` Series: value_counts](http://nullege.com/codes/search/pandas.Series.value_counts) for more detailed information.


<a id='common_operations'></a>

### Common Operations on Data by Type

---

- **`float`**: Precision-specific math operations.
- **`int`**: Operations with whole numbers.
- **`bool`**: Control flow conditions.
- **`datetime64`**: Resampling, slicing/selection, frequency back/front filling on a date range.
- **`timedelta`**: Date comparisons.
- **`category`**: A more powerful set type; can capture for example days as a category with ordinal (ordering) information.
- **`object`**: All data types can be represented as an object, but math and date operations will not be possible. Limited control flow possibilities are available unless you are comparing strings.

<a id='guided_practice'></a>

### Guided Practice: Inspecting Data Types and Applying Functions

---

[This guided practice](./practice-inspecting-data-applying-functions.ipynb) follows along with the questions in the Jupyter notebook provided.


In [1]:
import pandas as pd
import numpy as np

**1. Create a small DataFrame with different data types.**

In [2]:
test_data = dict( 
    A = np.random.rand(3),
    B = 1,
    C = 'foo',
    D = pd.Timestamp('20010102'),
    E = pd.Series([1.0]*3).astype('float32'),
    F = False,
    G = pd.Series([1]*3,dtype='int8')
)

In [3]:
test_data

{'A': array([ 0.52262705,  0.19537626,  0.89888063]),
 'B': 1,
 'C': 'foo',
 'D': Timestamp('2001-01-02 00:00:00'),
 'E': 0    1.0
 1    1.0
 2    1.0
 dtype: float32,
 'F': False,
 'G': 0    1
 1    1
 2    1
 dtype: int8}

In [4]:
dft = pd.DataFrame(test_data)
dft

,A,B,C,D,E,F,G
0,0.522627,1,foo,2001-01-02,1.0,False,1
1,0.195376,1,foo,2001-01-02,1.0,False,1
2,0.898881,1,foo,2001-01-02,1.0,False,1


**2. Examine the data types of the columns.**

In [5]:
dft.dtypes

A           float64
B             int64
C            object
D    datetime64[ns]
E           float32
F              bool
G              int8
dtype: object

**3. Create a Series object with integers 1-5 and float 6.0. What data type is the Series?**

In [6]:
pd.Series([1, 2, 3, 4, 5, 6.])

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
dtype: float64

If a `pandas` object contains data of multiple dtypes in a single column, the dtype of that column will be used to accommodate all data types (`object` is the most general).

**4. Create a Series with data `[1, 2, 3, 6., 'foo']`. What data type is the Series?**

In [7]:
pd.Series([1, 2, 3, 'foo'])

0      1
1      2
2      3
3    foo
dtype: object

**5. Use the `.get_dtype_counts()` function to determine how many columns there are of each type.**

In [5]:
dft.get_dtype_counts().astype(list)

bool              1
datetime64[ns]    1
float32           1
float64           1
int64             1
int8              1
object            1
dtype: object

**With a partner, take three minutes to discuss:**

*Without* running this code with a Python interpreter, what would you expect to be the most common `dtype`?

    [1, 3, 9, .33, False, '03-20-1978', np.arange(22)]



You can do a lot more with dtypes. For more information, check out 
[`pandas` Documentation](http://pandas.pydata.org/pandas-docs/stable/pandas.pdf).

**Applying Functions to Data with `df.apply()`**

Generally, `df.apply()` will apply a singlular function to every cell of the DataFrame it's used within.  

Note: There is another common built-in function, `df.map()`, that applies a function to each element of a single Series (column). For example:

```python
df['a'].map(my_func)
```

**6. Create another small DataFrame.**

In [6]:
# Create some more test data.
df = pd.DataFrame(np.random.randn(5, 4), columns=['a', 'b', 'c', 'd'])
df

,a,b,c,d
0,-0.410785,0.190240,1.745112,-0.322609
1,0.281392,-0.229490,1.962935,0.946956
2,0.809197,0.318424,1.247255,1.052985
3,0.858677,0.244996,0.090704,1.250873
4,-0.737841,0.966909,0.580455,-1.126967


**7. Use the `.apply()` function to find the square root of all cells.**

In [7]:
# Square root ALL CELLS (NaN == Not a Number).
df.apply(np.sqrt)

,a,b,c,d
0,NaN,0.436165,1.321027,NaN
1,0.530464,NaN,1.401048,0.973116
2,0.899554,0.564291,1.116806,1.026151
3,0.926648,0.494971,0.301172,1.118425
4,NaN,0.983315,0.761876,NaN


**8. Use `.apply()` to find the mean of the columns.**

In [8]:
df.apply(np.mean, axis=0)

a    0.160128
b    0.298216
c    1.125292
d    0.360248
dtype: float64

**9. Find the mean of the rows.**

In [9]:
df.apply(np.mean, axis=1)

0    0.300489
1    0.740448
2    0.856965
3    0.611313
4   -0.079361
dtype: float64

### Further Reading

For more advanced information on `.apply` usage, check out these links:

- ["Why Not"'s Gist Examples](https://gist.github.com/why-not/4582705)

- [Chris Albon's Map + Apply Examples](http://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html)


**Counting Occurrances of Unique Values With `.value_counts()`**

The `.value_counts` attribute tells us the count of unique values in a column's data. It's helpful for identifying unexpected values and getting a feel for the data's distribution, especially when looking at group membership.  Looking at the value counts per column can give us a quick overview of values expressed in our data.

Some common use cases of `.value_counts` include:
 - Finding strings inside of mostly numeric/continuous data.
 - Finding non-numeric values.
 - General distributions of categorical variables.
 - Identifying the most and least common values.

**10. Use `numpy` to create a random vector of 50 numbers ranging from 0 to 6.**


In [10]:
data = np.random.randint(0, 7, size = 50)
data

array([5, 5, 5, 2, 4, 2, 5, 4, 5, 4, 1, 1, 1, 3, 6, 1, 4, 6, 1, 0, 2, 2, 2,
       5, 6, 5, 1, 2, 4, 4, 5, 0, 4, 2, 2, 1, 1, 1, 0, 1, 3, 6, 2, 6, 3, 6,
       6, 6, 3, 6])

**11. Convert the vector to a Series and count the occurrences of each number.**

In [11]:
s = pd.Series(data)
s.head()

0    5
1    5
2    5
3    2
4    4
dtype: int64

In [12]:
# The counts of each number that occurs in our array is listed.
pd.value_counts(s)

1    10
6     9
2     9
5     8
4     7
3     4
0     3
dtype: int64

<a name="independent_ practice"></a>

### Independent Practice: Sales Data

---

1. Load the `sales.csv` data set from the `datasets` directory.
- Inspect the data types.
- Imagine you've found out that all your values in column 1 are off by one. Use `.apply()` or `.map()` to add `1` to column 1 of the data set.
- Use `.value_counts` to count the values of one column of the data set.
